In [1]:
from pyoptmat import ode, models, neuralode, solvers

import torch
from torch.func import vmap, jacfwd, jacrev

import numpy as np
import scipy

import matplotlib.pyplot as plt

torch.set_default_dtype(torch.DoubleTensor)

# Set up device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
import xarray as xr
from pyoptmat import experiments

# Load in the data
input_data = xr.open_dataset("data.nc")
data, results, cycles, types, control = experiments.load_results(
        input_data, device = device)

t = data[0]
stress = results.unsqueeze(dim = 1)
erate = data[-1].unsqueeze(dim = 1)
T = data[2].unsqueeze(dim = 1)
y = torch.rand((2,75,1000))

torch.cat(erate, T, dim = 1)

/home/gravisankar/packages/pyoptmat_tests/env/lib/python3.8/site-packages/xarray/backends/plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


RuntimeError: Index put requires the source and destination dtypes match, got Float for the destination and Double for the source.

In [2]:
class BackwardEuler(torch.nn.Module):
    def __init__(self, neural_ode, step_size, max_iter=100, tolerance=1e-6):
        super().__init__()
        self.neural_ode = neural_ode
        self.step_size = step_size
        self.max_iter = max_iter
        self.tolerance = tolerance

    def forward(self, t, y):
        y_next = y
        for _ in range(self.max_iter):
            y_next_new = y + self.step_size * self.neural_ode(t + self.step_size, y_next)
            if torch.norm(y_next_new - y_next) < self.tolerance:
                break
            y_next = y_next_new
        else:
            raise RuntimeError('Backward Euler method did not converge')
        return y_next_new 

In [3]:
class NeuralODE(torch.nn.Module):
    def __init__(self):
        super(NeuralODE, self).__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(5, 3),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(3, 3),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(3, 3),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(3, 3),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(3, 1)
        ).to(device)

    def forward(self, t, y):
        return self.net(y)

In [4]:
input_dim = 5
hidden_dim = 3
output_dim = 5
neural_ode = NeuralODE()

step_size = 0.01
backward_euler_solver = BackwardEuler(neural_ode, step_size)

# Define loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(neural_ode.parameters(), lr=0.01)

In [5]:
# Generate training data
t_values = torch.linspace(0, 2*np.pi, 100).to(device)
y_values = torch.sin(t_values).unsqueeze(-1).to(device)
y_values = y_values.repeat(1, 5)  # to match input_dim of NeuralODE


In [20]:
# Training loop
for epoch in range(2000):
    optimizer.zero_grad()
    t = torch.tensor(0.0)
    y_pred = backward_euler_solver(t, y_values[0])  # initial condition
    for y_true in y_values[1:]:
        t += step_size
        y_pred = backward_euler_solver(t, y_pred)
        loss = criterion(y_pred, y_true)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    if epoch % 200 == 0:
        print(f"Epoch {epoch}, Loss {loss.item()}")

# Generate test data
t_test = torch.linspace(0, 2*np.pi, 100)
y_test = torch.sin(t_test).unsqueeze(-1)
y_test = y_test.repeat(1, 5)  # to match input_dim of NeuralODE

# Test the model
t = torch.tensor(0.0)
y_pred = backward_euler_solver(t, y_test[0])  # initial condition
y_preds = [y_pred.detach().numpy()]
for y_true in y_test[1:]:
    t += step_size
    y_pred = backward_euler_solver(t, y_pred)
    y_preds.append(y_pred.detach().numpy())

y_preds = np.array(y_preds)

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot(t_test, y_test[:, 0], label='True function')
plt.plot(t_test, y_preds[:, 0], label='Neural ODE solution')
plt.legend()
plt.show()

/home/gravisankar/packages/pyoptmat_tests/env/lib/python3.8/site-packages/torch/autograd/__init__.py:200: UserWarning: Error detected in MulBackward0. Traceback of forward call that caused the error:
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/gravisankar/packages/pyoptmat_tests/env/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/gravisankar/packages/pyoptmat_tests/env/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/home/gravisankar/packages/pyoptmat_tests/env/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 736, in start
    self.io_loop.start()
  File "/home/gravisankar/packages/pyoptmat_tests/env/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 195, in

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.